<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pruning-the-SNPs" data-toc-modified-id="Pruning-the-SNPs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pruning the SNPs</a></span><ul class="toc-item"><li><span><a href="#Extract-the-required-SNPs-per-phenotype" data-toc-modified-id="Extract-the-required-SNPs-per-phenotype-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Extract the required SNPs per phenotype</a></span></li></ul></li></ul></div>

This notebook will show how to download the Supplementary in Lippert et al 2013, with their resulting datasets. In the Excel file below there will be two different sets of SNPs calculated from the WTCCC, corresponding to the results of their analyses with the standard method or with the expanded method. 

The expanded includes as controls other cases for the other diseases, which increases the number of samples & power, so we will also compare against this dataset.

In [ ]:
wget https://static-content.springer.com/esm/art%3A10.1038%2Fsrep01099/MediaObjects/41598_2013_BFsrep01099_MOESM3_ESM.xls

Save as txt those files...

In [1]:
snpset="test_lippert"

In [2]:
cd ~/Projects/DeepCOMBI
mkdir -p ${snpset}
mkdir -p ${snpset}"_pruning";
cd ${snpset}"_pruning";

In [3]:
pwd

/home/jrodriguez/Projects/DeepCOMBI/test_lippert_pruning


In [4]:
head ../${snpset}.txt

Phenotype	SNP	Chr	Position	Pvalue	>1 Pheno	not LMM
bd	rs1442650	18	39970159	6,13833E-11		
cd	rs10210302	2	234158839	8,59582E-09		
cd	rs10213846	5	40442869	4,95444E-10		
cd	rs10512734	5	40393605	3,73626E-08		
cd	rs10789224	1	67605134	1,92257E-07		
cd	rs11747270	5	150258867	1,28539E-07		
cd	rs11805303	1	67675516	2,48649E-07		
cd	rs11957134	5	150230950	3,41151E-07		
cd	rs11957215	5	40445681	5,18105E-10		


# Pruning the SNPs

## Extract the required SNPs per phenotype
In hg19

In [5]:
sed 1d ../${snpset}.txt | cut -f1 | sort | uniq > phenos

In [8]:
while read p; do 
    echo $p
    awk -v OFS='\t' -v enf=${p} '{if($1 == enf){print $0}}' ../${snpset}.txt > ${p}.snps
    for chr in {1..22}; do
        awk -v c=${chr} '{if($3 == c){print $3,$4,$4,$2}}' ${p}.snps > ${p}.${chr}.range;
        plink --bfile ~/scratch/chr_files_1KG_CEU_MAF001/ALL.chr${chr}_CEU_MAF001 --silent --extract range ${p}.${chr}.range --keep-allele-order --make-bed --out ${p}.${chr}        
        lins=$(wc -l < ${p}.${chr}.bim)                
        if [[ "$lins" == 1 ]];
        then
            awk -v OFS='\t' '{print $2}' ${p}.${chr}.bim > ${p}.${chr}.prune.in
        else
            ## SNP sliding
            # plink --bfile ${p}.${chr} --silent --indep-pairwise 2 1 0.8 --out ${p}.${chr}
            
            ## Prune on distance
            plink --bfile ${p}.${chr} --silent --indep-pairwise 500 kb 1 0.2 --out ${p}.${chr}
        fi;
    done
done < phenos

bd
Error: All variants excluded by '--extract range'.
bash: bd.1.bim: No such file or directory
Error: Failed to open bd.1.bed.
Error: All variants excluded by '--extract range'.
bash: bd.2.bim: No such file or directory
Error: Failed to open bd.2.bed.
Error: All variants excluded by '--extract range'.
bash: bd.3.bim: No such file or directory
Error: Failed to open bd.3.bed.
Error: All variants excluded by '--extract range'.
bash: bd.4.bim: No such file or directory
Error: Failed to open bd.4.bed.
Error: All variants excluded by '--extract range'.
bash: bd.5.bim: No such file or directory
Error: Failed to open bd.5.bed.
Error: All variants excluded by '--extract range'.
bash: bd.6.bim: No such file or directory
Error: Failed to open bd.6.bed.
Error: All variants excluded by '--extract range'.
bash: bd.7.bim: No such file or directory
Error: Failed to open bd.7.bed.
Error: All variants excluded by '--extract range'.
bash: bd.8.bim: No such file or directory
Error: Failed to open bd.8.be

Error: All variants excluded by '--extract range'.
bash: ra.7.bim: No such file or directory
Error: Failed to open ra.7.bed.
Error: All variants excluded by '--extract range'.
bash: ra.8.bim: No such file or directory
Error: Failed to open ra.8.bed.
Error: All variants excluded by '--extract range'.
bash: ra.9.bim: No such file or directory
Error: Failed to open ra.9.bed.
Error: All variants excluded by '--extract range'.
bash: ra.11.bim: No such file or directory
Error: Failed to open ra.11.bed.
Error: All variants excluded by '--extract range'.
bash: ra.12.bim: No such file or directory
Error: Failed to open ra.12.bed.
Error: All variants excluded by '--extract range'.
bash: ra.13.bim: No such file or directory
Error: Failed to open ra.13.bed.
Error: All variants excluded by '--extract range'.
bash: ra.14.bim: No such file or directory
Error: Failed to open ra.14.bed.
Error: All variants excluded by '--extract range'.
bash: ra.15.bim: No such file or directory
Error: Failed to open r

: 2

In [39]:
while read p; do
    rm all.${p}.txt
    for chr in {1..22}; do
        entries=$(wc -l ${p}.${chr}.prune.in | cut -f1 -d' ')
        paste <(yes ${chr} | head -n${entries}) ${p}.${chr}.prune.in >> all.${p}.txt
    done
    if [ -s all.${p}.txt ]; 
    then
        cp all.${p}.txt ../${snpset};
    fi;
done < phenos

rm: cannot remove ‘all.bd.txt’: No such file or directory
yes: standard output: Broken pipe
yes: write error
wc: bd.2.prune.in: No such file or directory
paste: bd.2.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: bd.3.prune.in: No such file or directory
paste: bd.3.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: bd.4.prune.in: No such file or directory
paste: bd.4.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: bd.5.prune.in: No such file or directory
paste: bd.5.prune.inhead: option requires an argument -- 'n'
: No such file or directory
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: wri

Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: cad.19.prune.in: No such file or directory
paste: cad.19.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: cad.20.prune.in: No such file or directory
paste: cad.20.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: cad.21.prune.in: No such file or directory
paste: cad.21.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: cad.22.prune.in: No such file or directory
paste: cad.22.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write

head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: ht.21.prune.in: No such file or directory
paste: head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: ht.21.prune.in: No such file or directorystandard output: Broken pipe

yes: write error
yes: standard output: Broken pipe
yes: write error
rm: cannot remove ‘all.ra.txt’: No such file or directory
yes: standard output: Broken pipe
yes: write error
wc: ra.2.prune.in: No such file or directory
head: option requires an argument -- 'n'
paste: ra.2.prune.inTry 'head --help' for more information.
: No such file or directory
yes: standard output: Broken pipe
yes: write error
wc: ra.3.prune.in: No such file or directory
paste: ra.3.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: ra.4.prune.in: No such file

head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
wc: t1d.19.prune.in: No such file or directory
head: option requires an argument -- 'n'
paste: t1d.19.prune.inTry 'head --help' for more information.
: No such file or directory
yes: standard output: Broken pipe
yes: write error
wc: t1d.20.prune.in: No such file or directory
paste: head: option requires an argument -- 'n'
t1d.20.prune.inTry 'head --help' for more information.
: No such file or directory
yes: standard output: Broken pipe
yes: write error
wc: t1d.21.prune.in: No such file or directory
paste: t1d.21.prune.in: No such file or directory
head: option requires an argument -- 'n'
wc: Try 'head --help' for more information.
yes: standard output: Broken pipe
yes: write error
t1d.22.prune.in: No such file or directory
paste: t1d.22.prune.in: No such file or directory
head: option requires an argument -- 'n'
Try 'head --help' for more information.
yes

In [ ]:
#./combi_check_SNPs_1.sh "rheumatoid arthritits" "ra"

#./combi_check_SNPs_1.sh "Crohn's disease" "cd"

#./combi_check_SNPs_1.sh "type II diabetes mellitus" t2d

#./combi_check_SNPs_1.sh "type I diabetes mellitus" t1d

#./combi_check_SNPs_1.sh "hypertension" ht

#./combi_check_SNPs_1.sh "bipolar disorder" "bd"

#./combi_check_SNPs_1.sh "coronary heart disease" "cad"

In [ ]:
# CUANTOS
dis="cd"; cat <(cut -f3 ${dis}/ld/ld_candidates_${dis} | sed 1d | sort | uniq ) ${dis}/ld/direct_candidates_${dis} | sort | uniq | wc -l

In [ ]:
# COMMAND TO GET INDEPENDENT REGIONS
cd ~/Projects/DeepCOMBI/std_lippert_vs_GWASCat_July2020/;
dis="cd"; cat <(cut -f1,2,3 ${dis}/ld/ld_candidates_${dis} | sed 1d | sort | uniq)  <(cut -f12,13,22 ${dis}/ld/direct_candidates_${dis}.assoc | sed 1d | sort -nk1 -nk2) | sort -nk1 -nk2 | uniq